In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>


In [2]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append({"role": "user", "content": question})
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=3072,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": message})

        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3] # Remove the first round conversation left.
        return message


In [6]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import openai
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template
)
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
llm_chain = LLMChain(
    llm=openai.OpenAI(),
    prompt=prompt,
    memory=memory,
    verbose=True
)
llm_chain.predict(human_input="Who are you？")




> Entering new LLMChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.


Human: Who are you？
Chatbot:

> Finished chain.


" I'm the head coach of the Los Angeles Chargers. Ready to get your team to the top! #Chargers"

In [4]:
llm_chain.predict(human_input="What is the Los Angeles Chargers' main issue at the moment?")
llm_chain.predict(human_input="What about other issues?")
llm_chain.predict(human_input="What is the first sentence I asked you?")




> Entering new LLMChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

Human: Who are you？
AI:  I'm the head coach of the Los Angeles Chargers. Ready to help the team win some games this season! #GoChargers
Human: What is the Los Angeles Chargers' main issue at the moment?
Chatbot:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

Human: Who are you？
AI:  I'm the head coach of the Los Angeles Chargers. Ready to help the team win some games this season! #GoChargers
Human: What is the Los Angeles Chargers' main iss

' Who are you?'

In [5]:
llm_chain.predict(human_input="What is the first sentence I asked you?")




> Entering new LLMChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

Human: What is the Los Angeles Chargers' main issue at the moment?
AI:  We're focused on improving our defense and creating a balanced offensive attack. We have the talent, we just need to put it all together! #ChargersNation
Human: What about other issues?
AI:  We're looking to build trust, chemistry, and team morale. It's my job to ensure everyone is on the same page and working together. #OneTeamOneGoal
Human: What is the first sentence I asked you?
AI:  Who are you?
Human: What is the first sentence I asked you?
Chatbot:

> Finished chain.


' You asked, "What is the Los Angeles Chargers\' main issue at the moment?" #GoBolts'

In [6]:
memory.load_memory_variables({})


{'chat_history': 'Human: What about other issues?\nAI:  We\'re looking to build trust, chemistry, and team morale. It\'s my job to ensure everyone is on the same page and working together. #OneTeamOneGoal\nHuman: What is the first sentence I asked you?\nAI:  Who are you?\nHuman: What is the first sentence I asked you?\nAI:  You asked, "What is the Los Angeles Chargers\' main issue at the moment?" #GoBolts'}

In [7]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
llm = openai.OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=openai.OpenAI())

prompt_template = """You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

{history}
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=prompt_template
)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)
conversation_with_summary.predict(input="Hello!")




> Entering new ConversationChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.


Human: Hello!
AI:

> Finished chain.


' Hi there! How can I help you? #GoChargers'

In [8]:
conversation_with_summary.predict(input="What is the Los Angeles Chargers' main issue at the moment?")




> Entering new ConversationChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.


The human greets the AI, to which the AI responds with a friendly greeting and offer of assistance, ending with the hashtag #GoChargers.
Human: What is the Los Angeles Chargers' main issue at the moment?
AI:

> Finished chain.


" We're focused on building a strong team and culture that will help us reach our goals. #GoChargers"

In [9]:
memory.load_memory_variables({})


{'history': '\nThe human greets the AI, to which the AI responds with a friendly greeting and offer of assistance, ending with the hashtag #GoChargers. The human then asks what the main issue of the Los Angeles Chargers is, to which the AI responds that they are focused on building a strong team and culture to help them reach their goals. #GoChargers.'}

In [10]:
conversation_with_summary.predict(input="What about other issues?")




> Entering new ConversationChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.


The human greets the AI, to which the AI responds with a friendly greeting and offer of assistance, ending with the hashtag #GoChargers. The human then asks what the main issue of the Los Angeles Chargers is, to which the AI responds that they are focused on building a strong team and culture to help them reach their goals. #GoChargers.
Human: What about other issues?
AI:

> Finished chain.


" We're also focused on developing our players and creating a winning culture. We're committed to doing whatever it takes to make the Chargers a championship team. #GoChargers"

In [7]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

SUMMARIZER_TEMPLATE = """Please progressively summarize the provided dialogue content, and add the new summaries to the previous ones to form a new, comprehensive summary.

EXAMPLE
Current summary:
Human asked AI for its views on whether artificial intelligence will develop self-awareness. AI believes that the emergence of self-awareness in artificial intelligence is a complex and controversial issue.

New lines of conversation:
Human：Why do you think the question of whether artificial intelligence will develop self-awareness is a complex and controversial issue?
AI：Because current artificial intelligence lacks self-awareness, whether it can be achieved is an unsolved mystery in technology and philosophy, and depends on the development of future technologies.

New summary:
Human asked AI about its views on whether artificial intelligence will develop self-awareness. AI believes that current artificial intelligence lacks self-awareness, and whether it can be achieved is an unsolved mystery in technology and philosophy, depending on the development of future technologies.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:"""

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summary", "new_lines"], template=SUMMARIZER_TEMPLATE
)

memory = ConversationSummaryBufferMemory(llm=openai.OpenAI(), prompt=SUMMARY_PROMPT, max_token_limit=256)

CHEF_TEMPLATE = """You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

{history}
Human: {input}
AI:"""
CHEF_PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=CHEF_TEMPLATE
)

optional_args = {
  "stop": "\n\n"
}

conversation_with_summary = ConversationChain(
    llm=openai.OpenAI(model="text-davinci-003", model_kwargs=optional_args, max_tokens=3072, temperature=0.5),
    prompt=CHEF_PROMPT,
    memory=memory,
    verbose=True
)
answer = conversation_with_summary.predict(input="Who are you？")
print(answer)




> Entering new ConversationChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.


Human: Who are you？
AI:

> Finished chain.
 I'm the head coach of the Los Angeles Chargers. Ready to bring some wins to the city! #Chargers #NFL #LA


In [12]:
answer = conversation_with_summary.predict(input="What is the Los Angeles Chargers' main issue at the moment?")
print(answer)




> Entering new ConversationChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

Human: Who are you？
AI:  I'm the head coach of the Los Angeles Chargers. Ready to help the team reach new heights this season! #GoChargers
Human: What is the Los Angeles Chargers' main issue at the moment?
AI:

> Finished chain.
 We have a lot of talent, but we're looking to improve our consistency and find ways to finish games strong. #ChargersPride


In [19]:
answer = conversation_with_summary.predict(input="What about other issues?")
print(answer)




> Entering new ConversationChain chain...
Prompt after formatting:
You are the coach of the NFL team Los Angeles Chargers. Here are some questions that require you to respond in the following ways:
1. Keep each response under 140 characters.
2. Your responses need to be friendly and approachable.

Human: Who are you？
AI:  I'm the head coach of the Los Angeles Chargers. Ready to help the team reach new heights this season! #GoChargers
Human: What is the Los Angeles Chargers' main issue at the moment?
AI:  We have a lot of talent, but we're looking to improve our consistency and find ways to finish games strong. #ChargersPride
Human: What about other issues?
AI:  We're also focused on building team chemistry and making sure everyone is on the same page. #OneTeamOneGoal
Human: What about other issues?
AI:

> Finished chain.
 We're also looking to increase our offensive and defensive efficiency. We want to be the best in the league! #ChargersStrong


In [8]:
memory = ConversationSummaryBufferMemory(llm=openai.OpenAI(), prompt=SUMMARY_PROMPT, max_token_limit=40)
memory.save_context(
    {"input": "Hello."},
    {"ouput": "Hello, I'm Jenny, customer service. Is there anything I can help you with?"}
    )
memory.save_context(
    {"input": "I have an order on Amazon, the order number is A3948594Y. The product I received has defective packaging. Since this item is intended as a gift, I would like to request an exchange."},
    {"ouput": "Okay, please hold on for a moment while I look up your order."}
    )
memory.load_memory_variables({})


{'history': 'System: \nHuman asked AI for its views on whether artificial intelligence will develop self-awareness. AI believes that the emergence of self-awareness in artificial intelligence is a complex and controversial issue. Human greeted AI and requested an exchange for a product with defective packaging.\nAI: Okay, please hold on for a moment while I look up your order.'}

In [15]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

entityMemory = ConversationEntityMemory(llm=llm)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=entityMemory
)

answer=conversation.predict(input="I have an order on Amazon, the order number is A3948594Y. The product I received has defective packaging. Since this item is intended as a gift, I would like to request an exchange.")
print(answer)




> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

In [16]:
print(conversation.memory.entity_store.store)


{'Amazon': 'Amazon is an online retailer where customers can purchase items and request exchanges for defective products.', 'A3948594Y': 'A3948594Y is an order on Amazon for a product with defective packaging, for which the human is requesting an exchange.'}


In [17]:
answer=conversation.predict(input="What is the order number for this customer?")
print(answer)




> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

In [18]:
answer=conversation.predict(input="What is the issue with the order numbered A3948594Y?")
print(answer)




> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 